In [ ]:
import os
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# matplotlib.use("agg")

from exp_spec_info import *
from plot_info import *
from select_data import *

In [ ]:
# Processed median path
median_data_path = "C:\\Users\\dosre\\dev\\thesis-data\\median_data.pkl"

# Matrix data
small_matrix_data_path = "C:\\Users\\dosre\\dev\\precision-cascade\\scripts\\python\\view_experimentation\\data\\small-experimental-matrix-data.csv"
large_matrix_data_path = "C:\\Users\\dosre\\dev\\precision-cascade\\scripts\\python\\view_experimentation\\data\\large-experimental-matrix-data.csv"

# Plot output directory
plot_output_root = "C:\\Users\\dosre\\dev\\thesis-data\\plots"
os.makedirs(plot_output_root, exist_ok=True)
plots_7_dir = os.path.join(plot_output_root, "plots_7")
os.makedirs(plots_7_dir, exist_ok=True)

In [ ]:
# Load matrix data
small_matrix_data = pd.read_csv(small_matrix_data_path).rename({"name": "matrix"}, axis=1)
large_matrix_data = pd.read_csv(large_matrix_data_path).rename({"name": "matrix"}, axis=1)

# Load data and filter for only gmres m solvers
median_data = pd.read_pickle(median_data_path)
median_data = median_data[median_data["solver"].transform(lambda x: (x in PC_SOLVERS))]

##### Prune Non-Complaint Experiments

In [ ]:
frac_err_compliance_threshold = 1.
compliant_data = median_data[median_data["med_rel_res_frac_err_fp64"] <= frac_err_compliance_threshold]
eliminated_data = median_data[median_data["med_rel_res_frac_err_fp64"] > frac_err_compliance_threshold]
mat_index_dict = dict(zip(
    eliminated_data["matrix"].unique(),
    np.arange(0, len(eliminated_data["matrix"].unique()))
))
index_mat_dict = dict(zip(
    np.arange(0, len(eliminated_data["matrix"].unique())),
    eliminated_data["matrix"].unique()
))

print(f"Valid data percentage: {len(compliant_data)/len(median_data):.2g}")
print(f"Eliminated data breakdown:")
print(eliminated_data["setup"].value_counts())
print(eliminated_data["restart_param"].value_counts())
print(eliminated_data["solver"].value_counts())
print(eliminated_data["matrix"].value_counts())

fig, ax = plt.subplots()

for solver in PC_SOLVERS:
    ax.scatter(
        eliminated_data["med_rel_res_frac_err_fp64"][eliminated_data["solver"] == solver],
        eliminated_data["matrix"][eliminated_data["solver"] == solver].transform(
            lambda mat: mat_index_dict[mat]
        ),
        marker=".",
        color=SOLVER_CLR_DICT[solver],
        label=solver
    )

ax.set_yticks(np.arange(0, len(eliminated_data["matrix"].unique())))
ax.set_yticklabels([index_mat_dict[int(label)] for label in np.arange(0, len(eliminated_data["matrix"].unique()))])
ax.semilogx()
ax.grid()
ax.legend()
ax.set_axisbelow(True)
ax.set_ylabel("Experiment")
ax.set_xlabel("Med. Rel. Res. Frac. Error to FP64")
ax.set_title(f"Rel. Res. Frac. Error Threshold $(>{frac_err_compliance_threshold})$ Non-Compliant Experiments")

plt.savefig(
    os.path.join(plots_7_dir, "threshold-non-compliance.png"),
    bbox_inches="tight"
)
plt.show()



##### 7.3 FP-GMRES and PC-GMRES Raw Comparison

In [ ]:
n_compliant_matrices = len(compliant_data["matrix"].unique())
mat_index_dict = dict(zip(
    compliant_data["matrix"].unique(), np.arange(0, n_compliant_matrices)
))
index_mat_dict = dict(zip(
    np.arange(0, n_compliant_matrices), compliant_data["matrix"].unique()
))

comparison_data = []

for restart_param in RESTART_PARAMS:

    restart_data = compliant_data[compliant_data["restart_param"] == restart_param]

    data_point = "med_rel_time_fp64"

    unprecond_data = restart_data[restart_data["setup"] == "unprecond"]
    unpreconddense_data = restart_data[restart_data["setup"] == "unpreconddense"]

    fig, axs = plt.subplots(
        1, len(PC_SOLVERS), figsize=(11.5, 8), sharey=True
    )
    x_range = [0., 3.]

    for ax, solver in zip(axs, PC_SOLVERS):
        for data, setup, marker, line, color in [
            (unprecond_data, "unprecond", ".", "--", SOLVER_CLR_DICT[solver]),
            (unpreconddense_data, "unpreconddense", "x", "--", SOLVER_SECONDARY_CLR_DICT[solver])
        ]:

            solver_data = data[data["solver"] == solver]

            ax.scatter(
                solver_data[data_point],
                solver_data["matrix"].transform(lambda mat: mat_index_dict[mat]),
                alpha=1.,
                color=color,
                zorder=2,
                marker=marker
            )
            ax.axvline(
                1.,
                color=SOLVER_CLR_DICT["FP FP64"],
                linestyle="solid",
                zorder=1
            )
            ax.axvline(
                solver_data[data_point].median(),
                color=color,
                linestyle=line,
                zorder=2
            )
            ax.set_title(f"{solver}")

            ax.set_xlim(x_range[0], x_range[1])

            comparison_data.append(
                {
                    "setup": setup,
                    "restart_param": restart_param,
                    "solver": solver,
                    data_point: solver_data[data_point].median()
                }
            )

        off_screen_data = pd.concat(
            [unprecond_data[unprecond_data[data_point] >= x_range[1]]["matrix"],
                unpreconddense_data[unpreconddense_data[data_point] >= x_range[1]]["matrix"]]
        ).value_counts()
        for matrix in off_screen_data.index:
            ax.text(
                x_range[0]+0.99*(x_range[1]-x_range[0]),
                mat_index_dict[matrix],
                f"{off_screen_data.loc[matrix]}$\\rightarrow$",
                verticalalignment='center',
                horizontalalignment='right'
            )

    for ax in axs:
        ax.grid()
    axs[3].set_xlabel(f"Median Relative Time")
    axs[0].set_yticks(np.arange(0, n_compliant_matrices))
    axs[0].set_yticklabels([index_mat_dict[label] for label in np.arange(0, n_compliant_matrices)])
    axs[0].set_ylabel("Experiment")

    fig.suptitle(f"Unpreconditioned PC-GMRES(${restart_param}$) Solver Relative Time to FP64")
    fig.tight_layout()

    plt.savefig(
        os.path.join(plots_7_dir, f"{restart_param:03d}-unprecond-rel-time.png")
    )
    plt.show()
    plt.close()

comparison_data = pd.DataFrame(comparison_data)


In [ ]:
unprecond = comparison_data[comparison_data["setup"] == "unprecond"]
unprecond = unprecond.drop("setup", axis=1)
unprecond = unprecond.pivot(index="solver", columns="restart_param", values="med_rel_time_fp64").loc[PC_SOLVERS[::-1]]
unprecond = unprecond.round(2)

unpreconddense = comparison_data[comparison_data["setup"] == "unpreconddense"]
unpreconddense = unpreconddense.drop("setup", axis=1)
unpreconddense = unpreconddense.pivot(index="solver", columns="restart_param", values="med_rel_time_fp64").loc[PC_SOLVERS[::-1]]
unpreconddense = unpreconddense.round(2)

max_val = max(unprecond.max().max(), unpreconddense.max().max())
min_val = min(unprecond.min().min(), unpreconddense.min().min())

fig, ax = plt.subplots()

solvers = unprecond.index.to_numpy()
restart_params = unprecond.columns.to_numpy()

X, Y = np.meshgrid(np.arange(0, len(restart_params)), np.arange(0, len(solvers)))
ax.pcolormesh(X, Y, unprecond.to_numpy(), cmap="RdYlGn_r", vmin=min_val, vmax=max_val)
for i in np.arange(0, len(restart_params)):
    for j in np.arange(0, len(solvers)):
        ax.text(i, j, unprecond.loc[solvers[j]][restart_params[i]], ha="center", va="center", fontsize=9)

ax.set_xticks(np.arange(0, len(restart_params)))
ax.set_xticklabels([restart_params[i] for i in np.arange(0, len(restart_params))])
ax.set_xlabel("Restart Parameter")
ax.set_yticks(np.arange(0, len(solvers)))
ax.set_yticklabels([solvers[i] for i in np.arange(0, len(solvers))])
ax.set_ylabel("PC-GMRES($m$) Solver")
ax.set_title("Unpreconditioned Sparse PC-GMRES($m$) Median Relative Time to FP64")

plt.savefig(
    os.path.join(plots_7_dir, "rel-time-summary-sparse.png"),
    bbox_inches="tight"
)
plt.show()

fig, ax = plt.subplots()

solvers = unpreconddense.index.to_numpy()
restart_params = unpreconddense.columns.to_numpy()

X, Y = np.meshgrid(np.arange(0, len(restart_params)), np.arange(0, len(solvers)))
ax.pcolormesh(X, Y, unpreconddense.to_numpy(), cmap="RdYlGn_r", vmin=min_val, vmax=max_val)
for i in np.arange(0, len(restart_params)):
    for j in np.arange(0, len(solvers)):
        ax.text(i, j, unpreconddense.loc[solvers[j]][restart_params[i]], ha="center", va="center", fontsize=9)

ax.set_xticks(np.arange(0, len(restart_params)))
ax.set_xticklabels([restart_params[i] for i in np.arange(0, len(restart_params))])
ax.set_xlabel("Restart Parameter")
ax.set_yticks(np.arange(0, len(solvers)))
ax.set_yticklabels([solvers[i] for i in np.arange(0, len(solvers))])
ax.set_ylabel("PC-GMRES($m$) Solver")
ax.set_title("Unpreconditioned Dense PC-GMRES($m$) Median Relative Time to FP64")

plt.savefig(
    os.path.join(plots_7_dir, "rel-time-summary-dense.png"),
    bbox_inches="tight"
)
plt.show()

##### 7.4

##### Misc. Analysis

In [ ]:
solver_data = median_data[
    (median_data["setup"] == "unprecond")
]

small_merge = solver_data.merge(small_matrix_data, on="matrix", how="inner")
large_merge = solver_data.merge(large_matrix_data, on="matrix", how="inner")

fig, ax = plt.subplots()
ax.scatter(small_merge["med_inner_iter"], small_merge["med_rel_time_fp64"], marker=".")
ax.scatter(large_merge["med_inner_iter"], large_merge["med_rel_time_fp64"], marker=".")
# ax.semilogx()
# ax.set_yscale("symlog")
print(np.corrcoef(
    np.hstack([small_merge["med_inner_iter"], large_merge["med_inner_iter"]]),
    np.hstack([small_merge["med_rel_time_fp64"], large_merge["med_rel_time_fp64"]])
))
plt.show()

fig, ax = plt.subplots()
ax.scatter(small_merge["nnz"], small_merge["med_rel_time_fp64"], marker=".")
ax.scatter(large_merge["nnz"], large_merge["med_rel_time_fp64"], marker=".")
# ax.semilogx()
# ax.set_yscale("symlog")
print(np.corrcoef(
    np.hstack([small_merge["nnz"], large_merge["nnz"]]),
    np.hstack([small_merge["med_rel_time_fp64"], large_merge["med_rel_time_fp64"]])
))
plt.show()

fig, ax = plt.subplots()
ax.scatter(small_merge["rows"], small_merge["med_rel_time_fp64"], marker=".")
ax.scatter(large_merge["rows"], large_merge["med_rel_time_fp64"], marker=".")
# ax.semilogx()
# ax.set_yscale("symlog")
print(np.corrcoef(
    np.hstack([small_merge["rows"], large_merge["rows"]]),
    np.hstack([small_merge["med_rel_time_fp64"], large_merge["med_rel_time_fp64"]])
))
plt.show()

fig, ax = plt.subplots()
ax.scatter(small_merge["cond"], small_merge["med_rel_time_fp64"], marker=".")
# ax.semilogx()
# ax.set_yscale("symlog")
print(np.corrcoef(small_merge["cond"], small_merge["med_rel_time_fp64"]))
plt.show()